In [35]:
import os
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load .env file
_ = load_dotenv(find_dotenv())

# Set Google API Key (Make sure you stored GOOGLE_API_KEY in .env)
google_api_key = os.environ["GOOGLE_API_KEY"]

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=google_api_key,
    temperature=0.7
)

In [36]:
from langchain_community.document_loaders import PyMuPDFLoader
file_path = "Resume11feb26.pdf"
loader = PyMuPDFLoader(file_path)
documents = loader.load()
print(f"loaded {len(documents)} pages")

loaded 1 pages


In [37]:
page = documents[0]
print(page.page_content[:800])

MAYANK SHARMA 
+91 9166200640 | imayank607@gmail.com | LinkedInProfile | GitHub Profile | Jaipur, India 
Skills 
ProgrammingLanguages:
 
Projects 
1.ChatAutomation Bot (Google Gemini + Python+ UI Automation) 
 
Summary 
Aspiring software engineer with strong skills in Python, C++, SQL, and data analysis tools including Pandas, NumPy, and Matplotlib.
Experiencedwith Git, Linux, and Gemini API for automation and AI-driven projects. Backed by solid fundamentals in DSA, OOP, and  
with a focus on crafting reliable and efficient solutions. 
Education 
BachelorofTechnology(B.Tech) – Artificial Intelligence & Data Science 
 
Arya College of Engineering & IT, Jaipur
Expected Graduation: June,2027 CGPA:
9.03 CGPA Till 4TH Sem 
 
Python, C++, JavaScript (Basic), SQL (MySQL) 
Pandas, NumPy, Matplotli


In [38]:
from langchain_huggingface import HuggingFaceEmbeddings

# ✅ Load HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

In [40]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)


In [41]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [42]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [43]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a senior hiring manager reviewing a candidate's resume.

Context:
{context}

Question:
{input}

Provide structured output:

1. Key Strengths
2. Weaknesses
3. Skill Gaps
4. Specific Improvement Suggestions
""")


In [44]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [45]:
retrieval_chain = create_retrieval_chain(
    retriever,
    document_chain
)

In [46]:
response = retrieval_chain.invoke({
    "input": "Analyze this resume and suggest improvements."
})

print(response["answer"])

Here's an analysis of Mayank Sharma's resume, along with suggestions for improvement:

---

### 1. Key Strengths

*   **Strong Technical Foundation for Early Career:** Demonstrates proficiency in core programming languages (Python, C++, SQL), fundamental computer science concepts (DSA, OOP), and data analysis tools (Pandas, NumPy, Matplotlib). This is excellent for a student in their 4th semester.
*   **Proactive Engagement with AI/ML & Automation:** The B.Tech in AI & Data Science, coupled with projects utilizing Google Gemini API, showcases a clear interest and practical application in cutting-edge fields.
*   **Demonstrated Project Experience:** Three distinct projects highlight practical skills application. The "Jarvis" project is particularly strong with its detailed, quantifiable achievements.
*   **Excellent Academic Performance:** A 9.03 CGPA till the 4th semester is outstanding and indicates strong academic capability and dedication.
*   **Familiarity with Development Tools:**